In [55]:
import datasets
from collections import Counter
import string
from gensim.utils import tokenize
from torch.utils.data import Dataset,DataLoader
from torch.nn import Module 
from torch import nn 
import torch

In [2]:
newsdata=datasets.load_dataset('ag_news')

In [16]:
#Создаём словарь количества вхождений слов
words=['<unk>','<bos>','<eos>','<pad>']
#str.maketrans('','',string.punctuation)-возврящяет словарь для замены
#.translate ждёт словарь слов для замены
for sent in newsdata['train']['text']:
    proced_sent=sent.lower().translate(
        str.maketrans('','',string.punctuation)
    )
    for word in tokenize(proced_sent):
        words.append(word)
        

vocab = set(['<unk>','<bos>','<eos>','<pad>'])#Токеный неизвестногого слова, начала,конца последовательности и токен пустного пропуска для батчей


treshold=25 #Порог для включения в словарь word2ind
words=Counter(words)

for word,cnt in words.items():
    if cnt>treshold:
        vocab.add(word)
print(f'Размер словаря {len(vocab)}')
        
   

Размер словаря 11505


In [17]:
word2ind={i:char for char,i in enumerate(vocab)}
ind2word={char:i for char,i in enumerate(vocab)}

In [45]:
class WordDataset(Dataset):
    def __init__(self,data):  
        super(WordDataset,self).__init__()
        self.data=data
        self.unk=word2ind['<unk>']
        self.bos=word2ind['<bos>']
        self.eos=word2ind['<eos>']
        self.pad=word2ind['<pad>']        
    def __getitem__(self,idx:int):
        #получаем оригинальные данные
        sent=self.data['text'][idx]
        label=self.data['label'][idx]
        proc_sent=sent.lower().translate(
            str.maketrans('','',string.punctuation)

        )
        #tokenized_sent=tokenize(proc_sent)
        tokenized_sent=[self.bos]
        tokenized_sent+=[
            word2ind.get(word,self.unk) for word in tokenize(proc_sent)
        ]
        tokenized_sent+=[self.eos]

        sample={
            'text':tokenized_sent,
            'label':label
        }
        return sample
    def __len__(self):
        return len(self.data)

In [50]:
def make_batch(data,max_len=256,pad_id=word2ind['<pad>']):
    lenghts=[len(sent['text']) for sent in data]
    max_len=min(max_len,max(lenghts))

    new_batch=[]
    for sent in data:
        sent['text']=sent['text'][:max_len]
        for i in range(max_len-len(sent['text'])):
            sent['text'].append(pad_id)
        new_batch.append(sent['text'])

    new_batch=torch.LongTensor(new_batch)

    new_pair={
        'text':torch.LongTensor(new_batch),
        'label':torch.LongTensor([x['label'] for x in data])
    }
    return new_pair

    

In [53]:
train_dataset=[newsdata['train']]

train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True,drop_last=True,collate_fn=make_batch)

In [57]:
class rnn_net(Module):
    def __init__(inp_size,hidden_size,out_size,vocab_size):
        super(rnn_net,self).__init__()

        self.emb=nn.Embedding(vocab_size,hidden_size)
        self.rnn=nn.RNN(hidden_size,hidden_size,batch_first=True)#!!!!!!!1
        self.fin_lin=nn.Linear(hidden_size,out_size)
        self.tahn=nn.Tanh()
    def forward(self,x):
        x=self.emb(x)
        x=self.tahn(self.rnn(x))

        #работа с эмбрендингами

        
        out=self.fin_lin(x)

        return x
        

In [58]:
model=rnn_net(256,256,4,len(vocab))

TypeError: rnn_net.__init__() takes 4 positional arguments but 5 were given

In [59]:
len(vocab)

11505